In [ ]:
import sys
print('Python Version %s' % sys.version)

In [ ]:
!pip list

In [ ]:
%%bash
conda list

In [ ]:
%%html

<iframe src="https://datascienceonaws.com" width="800px" height="600px"/>

# 1. Install the CLI tools

_Note:  Make sure the SageMaker role has access to create EKS Cluster._

In [ ]:
%%bash

source ~/.bash_profile

#### Install `eksctl`
# To get started we'll first install the `awscli` and `eksctl` CLI tools. [eksctl](https://eksctl.io) simplifies the process of creating EKS clusters.

pip install awscli --upgrade --user

curl --silent --location "https://github.com/weaveworks/eksctl/releases/latest/download/eksctl_$(uname -s)_amd64.tar.gz" | tar xz -C /tmp

sudo mv /tmp/eksctl /usr/local/bin

eksctl version

#### Install `kubectl`
# `kubectl` is a command line interface for running commands against Kubernetes clusters. 
# Run the following to install Kubectl

curl --location -o ./kubectl https://amazon-eks.s3.us-west-2.amazonaws.com/1.15.10/2020-02-22/bin/linux/amd64/kubectl

chmod +x ./kubectl

sudo mv ./kubectl /usr/local/bin

kubectl version --short --client

#### Install `aws-iam-authenticator`

curl -o aws-iam-authenticator https://amazon-eks.s3.us-west-2.amazonaws.com/1.15.10/2020-02-22/bin/linux/amd64/aws-iam-authenticator

chmod +x ./aws-iam-authenticator

sudo mv aws-iam-authenticator /usr/local/bin

aws-iam-authenticator version

#### Install jq and envsubst (from GNU gettext utilities) 
sudo yum -y install jq gettext

#### Verify the binaries are in the path and executable
for command in kubectl jq envsubst
  do
    which $command &>/dev/null && echo "$command in path" || echo "$command NOT FOUND"
  done

echo "Completed"

# 2. Configure the Environment Variables

In [ ]:
%%bash

source ~/.bash_profile

export AWS_REGION=$(aws configure get region)
echo "export AWS_REGION=${AWS_REGION}" | tee -a ~/.bash_profile

export AWS_CLUSTER_NAME=cluster
echo "export AWS_CLUSTER_NAME=${AWS_CLUSTER_NAME}" | tee -a ~/.bash_profile

echo "Completed"

# 3. Create the EKS Cluster

_Note:  Creating a cluster may take about 15 minutes.  Please be patient._

In [ ]:
%%bash

source ~/.bash_profile

cat << EOF > cluster.yaml
---
apiVersion: eksctl.io/v1alpha5
kind: ClusterConfig

metadata:
  name: ${AWS_CLUSTER_NAME}
  region: ${AWS_REGION}

cloudWatch:
  clusterLogging:
    enableTypes: ["*"]

managedNodeGroups:
- name: cpu-nodes
  instanceType: c5.xlarge
  volumeSize: 100
  desiredCapacity: 5
  iam:
    withAddonPolicies:
      albIngress: true

#secretsEncryption:
#  keyARN: ${MASTER_ARN}
EOF

In [ ]:
!cat cluster.yaml

In [ ]:
%%bash

source ~/.bash_profile

eksctl create cluster -f ./cluster.yaml

echo "Completed"

# _Please Wait 15-20 Minutes for the EKS Cluster to Start Successfully_

# _Your output will look like this..._

![](img/verify_eks.png)

# You will see retry errors. This is OK.